<a href="https://colab.research.google.com/github/fangyiyu/Fangyi_Yu_Queens/blob/master/Mnist_VGG_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, LSTM, BatchNormalization
from keras.utils import np_utils
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [2]:
!wget -O comp551w18-modified-mnist.zip https://www.dropbox.com/sh/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a?dl=0

--2020-09-28 20:48:24--  https://www.dropbox.com/sh/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a [following]
--2020-09-28 20:48:24--  https://www.dropbox.com/sh/raw/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce696705c370b8590c93d26b863.dl.dropboxusercontent.com/zip_download_get/Ajqw1oDuFSbPcFs__axFZ72OK4kbKUqvJnaICMhKPu9QLFCfnFeCBb7yXNHvv-d0jGJcRqZfiVFp3KlrkWvup9Ol0WejaFBmkIZ96s5khCNgHg [following]
--2020-09-28 20:48:24--  https://uce696705c370b8590c93d26b863.dl.dropboxusercontent.com/zip_download_get/Ajqw1oDuFSbPcFs__axFZ72OK4kbKUqvJnaICMhKPu9QLFCfnFeCBb7yXNHvv-d0jGJcRqZfiVFp3KlrkWvu

In [4]:
!unzip comp551w18-modified-mnist.zip

y
Archive:  comp551w18-modified-mnist.zip
  inflating: test_x.csv              
  inflating: train_y.csv             
  inflating: train_x.csv             


In [5]:
!ls

comp551w18-modified-mnist.zip  sample_data  train_x.csv
instruction.txt		       test_x.csv   train_y.csv


# 1. Data preprocessing

In [6]:
# Load data
x_train = np.loadtxt("train_x.csv", delimiter=",")# load from text
y_train = np.loadtxt("train_y.csv", delimiter=",")
x_test = np.loadtxt("test_x.csv", delimiter=",")

In [7]:
x_train=x_train.reshape(x_train.shape[0],64,64,1)
x_test=x_test.reshape(x_test.shape[0],64,64,1)

In [8]:
# Scale x
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 64, 64, 1)
50000 train samples
10000 test samples


In [10]:
# Split dataset into train and test
x_train_c, x_test_c, y_train_c, y_test_c = train_test_split(x_train, y_train, test_size=0.2, random_state=7)

In [11]:
# Transform y
n_classes = 10
y_train_d = np_utils.to_categorical(y_train_c, n_classes)
y_test_d = np_utils.to_categorical(y_test_c, n_classes)
print("Shape after one-hot encoding: ", y_train_d.shape)

Shape after one-hot encoding:  (40000, 10)


In [12]:
(x_train_c.shape, x_test_c.shape,y_train_d.shape,y_test_d.shape)

((40000, 64, 64, 1), (10000, 64, 64, 1), (40000, 10), (10000, 10))

In [14]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20, 
    width_shift_range=0.1,
    shear_range=0.2,
    height_shift_range=0.1,
    zoom_range=0.2
    )

datagen.fit(x_train_c)

In [15]:
input_shape = (64,64,1)

## 2. CNN_VGG

In [21]:
# Build model
def VGG_modified(input_shape, opt = tf.keras.optimizers.Adam(amsgrad=True)):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3), padding="same", activation='relu', input_shape=input_shape,
                            data_format="channels_last"))
    model.add(Conv2D(32, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(Conv2D(256, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    #sgd=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [22]:
model = VGG_modified((64,64,1))
history = model.fit_generator(datagen.flow(x_train_c, y_train_d, batch_size=64),
                              validation_data=(x_test_c, y_test_d),
                              epochs = 10)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
625/625 [==============================] - 30s 49ms/step - loss: 2.2434 - accuracy: 0.1729 - val_loss: 2.4569 - val_accuracy: 0.0692
Epoch 2/10
625/625 [==============================] - 29s 47ms/step - loss: 2.0054 - accuracy: 0.2489 - val_loss: 2.4053 - val_accuracy: 0.1220
Epoch 3/10
625/625 [==============================] - 30s 48ms/step - loss: 1.5995 - accuracy: 0.4115 - val_loss: 1.1985 - val_accuracy: 0.5427
Epoch 4/10
625/625 [==============================] - 30s 49ms/step - loss: 1.1276 - accuracy: 0.6140 - val_loss: 1.0224 - val_accuracy: 0.6734
Epoch 5/10
625/625 [==============================] - 31s 49ms/step - loss: 0.7080 - accuracy: 0.8029 - val_loss: 0.3645 - val_accuracy: 0.9034
Epoch 6/10
625/625 [==============================] - 30s 49ms/step - loss: 0.5274 - accuracy: 0.8569 - val_loss: 0.2940 - val_accuracy: 0.9174
Epoch 7/10
625/625 [==============================] - 30s 48

# 3. RNN_LSTM

In [23]:
# Hyper parameters
batch_size = 64
nb_epoch = 10
# Parameters for MNIST dataset
img_rows, img_cols = 64, 64
nb_classes = 10
# Parameters for LSTM network
nb_lstm_outputs = 30
nb_time_steps = img_rows
dim_input_vector = img_cols
input_shape = (64, 64)

x_train_r=x_train_c.reshape(x_train_c.shape[0],64,64)
x_test_r=x_test_c.reshape(x_test_c.shape[0],64,64)

model_LSTM = Sequential()
model_LSTM.add(LSTM(nb_lstm_outputs, input_shape=input_shape))
model_LSTM.add(Dense(10, activation='softmax'))
model_LSTM.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model_LSTM.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30)                11400     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                310       
Total params: 11,710
Trainable params: 11,710
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Train the model
history = model_LSTM.fit(x_train_r, 
                    y_train_d, 
                    epochs=nb_epoch, 
                    batch_size=batch_size, 
                    shuffle=True,
                    validation_data=(x_test_r, y_test_d),
                    verbose=1)

Epoch 1/10
625/625 [==============================] - 5s 7ms/step - loss: 2.3042 - accuracy: 0.1083 - val_loss: 2.2987 - val_accuracy: 0.1176
Epoch 2/10
625/625 [==============================] - 4s 7ms/step - loss: 2.2826 - accuracy: 0.1367 - val_loss: 2.2627 - val_accuracy: 0.1502
Epoch 3/10
625/625 [==============================] - 4s 7ms/step - loss: 2.2469 - accuracy: 0.1567 - val_loss: 2.2485 - val_accuracy: 0.1561
Epoch 4/10
625/625 [==============================] - 4s 6ms/step - loss: 2.2229 - accuracy: 0.1702 - val_loss: 2.2173 - val_accuracy: 0.1686
Epoch 5/10
625/625 [==============================] - 4s 6ms/step - loss: 2.2106 - accuracy: 0.1786 - val_loss: 2.2099 - val_accuracy: 0.1772
Epoch 6/10
625/625 [==============================] - 4s 6ms/step - loss: 2.1974 - accuracy: 0.1820 - val_loss: 2.2031 - val_accuracy: 0.1790
Epoch 7/10
625/625 [==============================] - 4s 7ms/step - loss: 2.1881 - accuracy: 0.1840 - val_loss: 2.1892 - val_accuracy: 0.1818
Epoch 